# Excel Data Extraction and Cleaning for ML Models

This notebook demonstrates how the `extract_files.py` script loads Excel files and cleans them for use in ML inference.

## Overview

The `extract_files.py` script provides functionality to:
1. Read Excel files (.xlsx/.xls) with multiple engine fallbacks
2. Select specific columns by name or index
3. Handle missing values (drop or fill)
4. Convert to NumPy arrays
5. Apply optional scaling (standard or minmax)
6. Save processed data as .npz files with metadata

In [ ]:
# Import necessary libraries
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the Processing directory to the path
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

# Reload the module to get the latest changes
import importlib
if 'extract_files' in sys.modules:
    importlib.reload(sys.modules['extract_files'])

# Import functions from extract_files
from extract_files import (
    _read_excel_with_fallbacks,
    _fix_excel_headers,
    _select_columns,
    _handle_missing,
    _scale_array,
    excel_to_numpy
)

print("✅ Module loaded with header fix!")

## Step 1: Set the File Path

For this example, we'll use a placeholder filename. Replace this with an actual file path from your data.

In [9]:
# Set the filename - replace with actual file path
# FILENAME = "../Data/Processed Data/Extract Data 480V/KINETRICS/08-5103VI.xlsx"

# For demonstration, let's use an actual file from the project
# Uncomment one of these lines to use real data:
FILENAME = "../Data/Processed Data/Extract Data 0.6kV Kinetrics/K-016084-677a.xls"
# FILENAME = "../Data/Processed Data/Extract Data 480V/some_file.xls"

print(f"Target file: {FILENAME}")

Target file: ../Data/Processed Data/Extract Data 0.6kV Kinetrics/K-016084-677a.xls


## Step 2: Read Excel File

The `_read_excel_with_fallbacks` function tries multiple engines to read the Excel file.

In [ ]:
# Read the Excel file
# This will return a dictionary of {sheet_name: DataFrame}
try:
    sheets_dict = _read_excel_with_fallbacks(FILENAME, sheet=None)
    print(f"Successfully read {len(sheets_dict)} sheet(s)")
    print(f"Sheet names: {list(sheets_dict.keys())}")
    
    # Apply header fix to each sheet
    print("\n" + "="*60)
    print("FIXING HEADERS")
    print("="*60)
    
    sheets_dict_fixed = {}
    for sheet_name, df in sheets_dict.items():
        print(f"\n--- {sheet_name} ---")
        print(f"Before fix - Columns: {list(df.columns[:5])}")
        print(f"Before fix - Shape: {df.shape}")
        
        df_fixed = _fix_excel_headers(df)
        sheets_dict_fixed[sheet_name] = df_fixed
        
        print(f"After fix - Columns: {list(df_fixed.columns[:5])}")
        print(f"After fix - Shape: {df_fixed.shape}")
        
        if not df_fixed.empty:
            print(f"\nFirst few rows with CORRECT column names:")
            display(df_fixed.head())
    
    # Update the main dictionary
    sheets_dict = sheets_dict_fixed
    
except Exception as e:
    print(f"Error reading file: {e}")
    import traceback
    traceback.print_exc()
    print("\nPlease update FILENAME with a valid Excel file path.")

Successfully read 3 sheet(s)
Sheet names: ['Sheet1', 'Sheet2', 'Sheet3']

FIXING HEADERS

--- Sheet1 ---
Before fix - Columns: ['Recording title: ', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Before fix - Shape: (10005, 10)
Error reading file: name 'fix_excel_headers' is not defined

Please update FILENAME with a valid Excel file path.


Traceback (most recent call last):
  File "/var/folders/_6/sh39ykv94sz7qqnm5qywwbzm0000gn/T/ipykernel_45000/3438610182.py", line 19, in <module>
    df_fixed = fix_excel_headers(df)
               ^^^^^^^^^^^^^^^^^
NameError: name 'fix_excel_headers' is not defined


## Step 3: Select and Clean Data

Let's work with the first sheet and demonstrate various cleaning operations.

In [11]:
# Select the first sheet
sheet_name = list(sheets_dict.keys())[0]
df = sheets_dict[sheet_name].copy()

print(f"Working with sheet: {sheet_name}")
print(f"Original shape: {df.shape}")
print(f"\nData types:")
print(df.dtypes)
print(f"\nMissing values:")
print(df.isnull().sum())

Working with sheet: Sheet1
Original shape: (10005, 10)

Data types:
Recording title:     object
Unnamed: 1           object
Unnamed: 2           object
Unnamed: 3           object
Unnamed: 4           object
Unnamed: 5           object
Unnamed: 6           object
Unnamed: 7           object
Unnamed: 8           object
Unnamed: 9           object
dtype: object

Missing values:
Recording title:     0
Unnamed: 1           0
Unnamed: 2           3
Unnamed: 3           3
Unnamed: 4           3
Unnamed: 5           3
Unnamed: 6           3
Unnamed: 7           3
Unnamed: 8           3
Unnamed: 9           3
dtype: int64


### Step 3.1: Select Only Numeric Columns

In [12]:
# Select only numeric columns
df_numeric = _select_columns(df, columns=None, numeric_only=True)

print(f"After selecting numeric columns: {df_numeric.shape}")
print(f"Numeric columns: {list(df_numeric.columns)}")
display(df_numeric.head())

After selecting numeric columns: (10005, 0)
Numeric columns: []


""
0
1
2
3
4


### Step 3.2: Handle Missing Values

We can either drop rows with missing values or fill them with a specific value.

In [13]:
# Option 1: Drop rows with any missing values
df_no_na = _handle_missing(df_numeric, dropna=True, fillna=None)
print(f"After dropping NaN rows: {df_no_na.shape}")

# Option 2: Fill missing values with 0 (or any other value)
df_filled = _handle_missing(df_numeric, dropna=False, fillna=0.0)
print(f"After filling NaN with 0: {df_filled.shape}")

# Use the no-NA version for further processing
df_clean = df_no_na.copy()

After dropping NaN rows: (10005, 0)
After filling NaN with 0: (10005, 0)


## Step 4: Convert to NumPy Array

In [14]:
# Convert to NumPy array
array_unscaled = df_clean.to_numpy()

print(f"NumPy array shape: {array_unscaled.shape}")
print(f"Data type: {array_unscaled.dtype}")
print(f"\nArray statistics:")
print(f"Min: {np.min(array_unscaled, axis=0)}")
print(f"Max: {np.max(array_unscaled, axis=0)}")
print(f"Mean: {np.mean(array_unscaled, axis=0)}")
print(f"Std: {np.std(array_unscaled, axis=0)}")

NumPy array shape: (10005, 0)
Data type: float64

Array statistics:
Min: []
Max: []
Mean: []
Std: []


## Step 5: Apply Scaling

Scaling is important for ML models. We can apply:
- **Standard scaling**: (x - mean) / std
- **MinMax scaling**: (x - min) / (max - min)
- **None**: No scaling

### Standard Scaling

In [17]:
# Apply standard scaling
array_standard, params_standard = _scale_array(array_unscaled, mode="standard")

print("Standard Scaling Applied")
print(f"Scaled array shape: {array_standard.shape}")
print(f"\nScaling parameters (mean, std):")
for col_idx, (mean, std) in params_standard.items():
    col_name = df_clean.columns[int(col_idx)] if int(col_idx) < len(df_clean.columns) else f"Col_{col_idx}"
    print(f"  {col_name}: mean={mean:.4f}, std={std:.4f}")

print(f"\nScaled array statistics:")
print(f"Mean (should be ~0): {np.mean(array_standard, axis=0)}")
print(f"Std (should be ~1): {np.std(array_standard, axis=0)}")

Standard Scaling Applied
Scaled array shape: (10005, 0)

Scaling parameters (mean, std):

Scaled array statistics:
Mean (should be ~0): []
Std (should be ~1): []


### MinMax Scaling

In [18]:
# Apply minmax scaling
array_minmax, params_minmax = _scale_array(array_unscaled, mode="minmax")

print("MinMax Scaling Applied")
print(f"Scaled array shape: {array_minmax.shape}")
print(f"\nScaling parameters (min, max):")
for col_idx, (min_val, max_val) in params_minmax.items():
    col_name = df_clean.columns[int(col_idx)] if int(col_idx) < len(df_clean.columns) else f"Col_{col_idx}"
    print(f"  {col_name}: min={min_val:.4f}, max={max_val:.4f}")

print(f"\nScaled array statistics:")
print(f"Min (should be 0): {np.min(array_minmax, axis=0)}")
print(f"Max (should be 1): {np.max(array_minmax, axis=0)}")

MinMax Scaling Applied
Scaled array shape: (10005, 0)

Scaling parameters (min, max):

Scaled array statistics:
Min (should be 0): []
Max (should be 1): []


## Step 6: Visualize Scaling Effects

In [20]:
# Visualize the effect of scaling on the first column
if array_unscaled.shape[1] > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Original data
    axes[0].hist(array_unscaled[:, 0], bins=20, edgecolor='black')
    axes[0].set_title('Original Data (First Column)')
    axes[0].set_xlabel('Value')
    axes[0].set_ylabel('Frequency')
    
    # Standard scaled
    axes[1].hist(array_standard[:, 0], bins=20, edgecolor='black', color='orange')
    axes[1].set_title('Standard Scaled')
    axes[1].set_xlabel('Value')
    axes[1].set_ylabel('Frequency')
    
    # MinMax scaled
    axes[2].hist(array_minmax[:, 0], bins=20, edgecolor='black', color='green')
    axes[2].set_title('MinMax Scaled')
    axes[2].set_xlabel('Value')
    axes[2].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

## Step 7: Using the Complete Pipeline

The `excel_to_numpy` function performs all these steps automatically and saves the results.

In [21]:
# Use the complete pipeline
output_dir = "./processed_data_output"

try:
    archive_path = excel_to_numpy(
        paths=[FILENAME],
        sheet=None,  # Process all sheets
        header=None,  # Use default header
        skiprows=0,
        columns=None,  # Use all columns
        numeric_only=True,  # Keep only numeric columns
        dropna=True,  # Drop rows with missing values
        fillna=None,
        scale="standard",  # Apply standard scaling
        output_dir=output_dir,
        save_meta=True
    )
    
    print(f"Data saved to: {archive_path}")
    
    # Load the saved data
    loaded_data = np.load(archive_path)
    print(f"\nArrays in archive: {list(loaded_data.keys())}")
    
    # Load metadata
    meta_path = os.path.join(output_dir, "excel_arrays.metadata.json")
    with open(meta_path, 'r') as f:
        metadata = json.load(f)
    
    print(f"\nMetadata:")
    print(json.dumps(metadata, indent=2))
    
except Exception as e:
    print(f"Error in pipeline: {e}")
    print("This is expected if FILENAME is not a valid file path.")

Data saved to: ./processed_data_output/excel_arrays.npz

Arrays in archive: ['K-016084-677a::Sheet1', 'K-016084-677a::Sheet2', 'K-016084-677a::Sheet3']

Metadata:
{
  "K-016084-677a::Sheet1": {
    "source_file": "/Users/aarijatiq/Documents/GitHub/Schneider_Arc_Flash_Prediction/Data/Processed Data/Extract Data 0.6kV Kinetrics/K-016084-677a.xls",
    "sheet": "Sheet1",
    "shape": [
      10005,
      0
    ],
    "columns": [],
    "scale_mode": "standard",
    "scale_params": {},
    "dropna": true,
    "fillna": null,
    "numeric_only": true
  },
  "K-016084-677a::Sheet2": {
    "source_file": "/Users/aarijatiq/Documents/GitHub/Schneider_Arc_Flash_Prediction/Data/Processed Data/Extract Data 0.6kV Kinetrics/K-016084-677a.xls",
    "sheet": "Sheet2",
    "shape": [
      0,
      0
    ],
    "columns": [],
    "scale_mode": "standard",
    "scale_params": {},
    "dropna": true,
    "fillna": null,
    "numeric_only": true
  },
  "K-016084-677a::Sheet3": {
    "source_file": "/Users

## Step 8: Load Processed Data for ML Model

Once the data is processed and saved, you can easily load it for ML model training or inference.

In [22]:
# Example: Load processed data for ML inference
def load_processed_data_for_ml(archive_path, metadata_path):
    """
    Load processed data ready for ML model inference.
    
    Returns:
        dict: Dictionary with array names as keys and processed arrays as values
        dict: Metadata for each array
    """
    # Load arrays
    data = np.load(archive_path)
    arrays = {key: data[key] for key in data.keys()}
    
    # Load metadata
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    return arrays, metadata

# Example usage
if os.path.exists(output_dir):
    try:
        arrays, metadata = load_processed_data_for_ml(
            os.path.join(output_dir, "excel_arrays.npz"),
            os.path.join(output_dir, "excel_arrays.metadata.json")
        )
        
        print("Loaded processed data for ML:")
        for key, arr in arrays.items():
            print(f"\n{key}:")
            print(f"  Shape: {arr.shape}")
            print(f"  Scaling: {metadata[key]['scale_mode']}")
            print(f"  Columns: {metadata[key]['columns']}")
            print(f"  Ready for ML model inference: Yes ✓")
    except Exception as e:
        print(f"Could not load data: {e}")
else:
    print(f"Output directory '{output_dir}' does not exist.")
    print("Run the pipeline with a valid Excel file first.")

Loaded processed data for ML:

K-016084-677a::Sheet1:
  Shape: (10005, 0)
  Scaling: standard
  Columns: []
  Ready for ML model inference: Yes ✓

K-016084-677a::Sheet2:
  Shape: (0, 0)
  Scaling: standard
  Columns: []
  Ready for ML model inference: Yes ✓

K-016084-677a::Sheet3:
  Shape: (0, 0)
  Scaling: standard
  Columns: []
  Ready for ML model inference: Yes ✓


## Summary

This notebook demonstrated:

1. **Reading Excel files** with multiple engine fallbacks for compatibility
2. **Selecting columns** by name or index, filtering for numeric data
3. **Handling missing values** through dropping or filling
4. **Converting to NumPy arrays** for ML compatibility
5. **Applying scaling** (standard or minmax) for feature normalization
6. **Saving processed data** as .npz files with metadata
7. **Loading data** for ML model training/inference

### Command-line Usage

You can also use the script from the command line:

```bash
# Process a single file
python extract_files.py path/to/file.xlsx --numeric-only --dropna --scale standard

# Process multiple files
python extract_files.py file1.xlsx file2.xls --numeric-only --scale minmax --output-dir ./output

# Process all files in a directory
python extract_files.py ../Data/Processed\ Data/ --numeric-only --dropna --scale standard
```

### Next Steps

- Replace `FILENAME` with actual Excel files from your dataset
- Adjust scaling method based on your ML model requirements
- Use the processed .npz files for model training or inference
- Refer to the metadata JSON for information about the preprocessing applied